In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
class HouseDataSet(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = pd.read_csv(data)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data.iloc[idx, :-1], self.data.iloc[idx, -1].item()

In [3]:
full_dataset = HouseDataSet("data")

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [0.95, 0.05])

In [5]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True)
val_dataset = torch.utils.data.DataLoader(val_dataset)

In [7]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(287, 100)
        self.linear2 = nn.Linear(100, 100)
        self.linear3 = nn.Linear(100, 1)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.activation(self.linear1(x))
        x = self.activation(self.linear2(x))
        x = self.activation(self.linear3(x))
        return x
    

In [8]:
model = Network()

In [9]:
loss_fn = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

In [11]:
epochs = 5

for epoch in range(epochs):
    loss_val = 0
    for house, price in (pbar := tqdm(train_dataloader)):
        optimizer.zero_grad()
        pred = model(house)
        loss = loss_fn(pred, price)
        loss.backward()
        loss_val += loss.item()
        
        optimizer.step()
        
        pbar.set_description(f"loss: {loss.item():.5f}")

loss: 1.40576: 100%|██████████| 3750/3750 [01:38<00:00, 37.97it/s]


1.562981344850858


loss: 0.73487: 100%|██████████| 3750/3750 [01:51<00:00, 33.77it/s]


1.1037349614898364


loss: 0.21521: 100%|██████████| 3750/3750 [01:14<00:00, 50.40it/s] 


0.7472377012009422


loss: 1.10486: 100%|██████████| 3750/3750 [01:25<00:00, 43.96it/s] 


0.6867399440020323


loss: 0.77737: 100%|██████████| 3750/3750 [01:28<00:00, 42.60it/s]

0.653815554845333


In [12]:
print(model)

NeuralNumbers(
  (flat): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=784, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
  (activation): ReLU()
)
